# Content Translation Article Deletion Ratios, across all wikis [FY 23 - Q4]
**Krishna Chaitanya Velaga, Data Scientist III, Wikimedia Foundation**

**Last updated on 16 September 2023**

[TASK: T343300](https://phabricator.wikimedia.org/T343300)

<u>Attribution:</u> This analysis has been built upon the [previous iteration of the analysis](https://gitlab.wikimedia.org/mneisler/content_translation_deletion_stats/-/blob/master/content_translation_deletion_ratios.ipynb) (written in R) by [Megan Neilser](https://github.com/MeganNeisler).

# Contents

1. [Overview](#Overview)
2. [Data Gathering](#Data-Gathering)
3. [Analysis](#Analysis)
    * [Current Quarter - FY23 Q4](#Current-Quarter)
    * [Previous Quarter - FY23 Q3](#Previous-Quarter)

# Overview

## Purpose
The purpose of this analysis is to identify and list the number of wikis where the deletion rate of articles created with content translation is higher than the deletion rate for articles created with other tools. Specifically, we want to answer the following questions:
* How many wikis have translations deleted more often than regular articles?
* Which are these wikis?
* Has the number of those wikis reduced compared to the previous period?
* How high is the highest deletion ratio a wiki has for translations?
* This analysis will be used as a baseline to assess the evolution of deletion rates as improvements are made.

## Summary
PLACEHOLDER

# Data-Gathering

In [217]:
import numpy as np
import pandas as pd
import wmfdata as wmf

pd.options.display.max_columns = None

import warnings

## spark_session

In [3]:
spark_session = wmf.spark.get_active_session()

if type(spark_session) != type(None):
    spark_session.stop()
else:
    print('no active session')

no active session


In [5]:
spark_session = wmf.spark.create_custom_session(
    master="yarn",
    app_name='cx-deletion-stats',
    spark_config={
        "spark.driver.memory": "4g",
        "spark.dynamicAllocation.maxExecutors": 64,
        "spark.executor.memory": "16g",
        "spark.executor.cores": 4,
        "spark.sql.shuffle.partitions": 256,
        "spark.driver.maxResultSize": "2g"
        
    }
)

SPARK_HOME: /usr/lib/spark3
Using Hadoop client lib jars at 3.2.0, provided by Spark.
PYSPARK_PYTHON=/opt/conda-analytics/bin/python3


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/16 06:29:23 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/09/16 06:29:23 WARN Utils: Service 'sparkDriver' could not bind on port 12000. Attempting port 12001.
23/09/16 06:29:23 WARN Utils: Service 'sparkDriver' could not bind on port 12001. Attempting port 12002.
23/09/16 06:29:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/09/16 06:29:24 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/09/16 06:29:34 WARN Utils: Service 'org.apache.spark.network.netty.NettyBlockTransferService' could not bind on port 13000. Attempting port 13001.
23/09/16 06:29:34 WARN Utils: Service 'org.apache.spark.network.netty.NettyBlockTransferService' could not bind on 

In [6]:
spark_session

In [7]:
spark_session.sparkContext.setLogLevel("ERROR")

## query

In [218]:
# set time period variables for each quarter

currq = dict(
    mw_snapshot = '2023-07',
    start_dt = '2023-04-01',
    end_dt = '2023-06-30'
)

prevq = dict(
    mw_snapshot = '2023-04',
    start_dt = '2023-01-01',
    end_dt = '2023-03-31'
)

In [219]:
deletion_stats_query = """
WITH base_counts AS (
    SELECT
        wiki_db,
        SUM(CASE WHEN ARRAY_CONTAINS(revision_tags, 'contenttranslation') THEN 1 ELSE 0 END) AS created_cx,
        SUM(CASE WHEN ARRAY_CONTAINS(revision_tags, 'contenttranslation') AND revision_is_deleted_by_page_deletion = TRUE THEN 1 ELSE 0 END) AS deleted_cx,
        SUM(CASE WHEN revision_is_deleted_by_page_deletion = TRUE THEN 1 ELSE 0 END) AS total_deleted,
        COUNT(*) AS total_articles
    FROM wmf.mediawiki_history
    WHERE
        snapshot = '{MW_SNAPSHOT}'
        AND event_timestamp BETWEEN '{START_DATE}' and '{END_DATE}'
        -- article namespace only
        AND page_namespace = 0
        -- new page creations only
        AND revision_parent_id = 0
        AND event_entity = 'revision'
        AND event_type = 'create'
        -- remove bots
        AND size(event_user_is_bot_by) <= 0 
    GROUP BY  
        wiki_db
)

SELECT
    wiki_db,
    created_cx,
    total_articles - created_cx AS created_non_cx,
    deleted_cx,
    total_deleted - deleted_cx AS deleted_non_cx
FROM base_counts
"""

In [225]:
# function to run a query to get deletion stats of a given quarter
def query_deletion_stats(quarter: str, query=deletion_stats_query):
    
    if quarter == 'current':
        mediawiki_snapshot = currq['mw_snapshot']
        start_date = currq['start_dt']
        end_date = currq['end_dt']
    elif quarter == 'previous':
        mediawiki_snapshot = prevq['mw_snapshot']
        start_date = prevq['start_dt']
        end_date = prevq['end_dt']
        
    formatted_query = query.format(
        MW_SNAPSHOT=mediawiki_snapshot,
        START_DATE=start_date, 
        END_DATE=end_date
    )
    
    return wmf.spark.run(formatted_query)

# function to calculate overall deletion ratio, and print explantory statements if needed
def overall_deletion_pct(df: pd.DataFrame, period: str, filter_threshold=15, pr=False):
    
    df_filtered = df.query("created_cx > @filter_threshold")
    
    deleted_cx_ratio = round(df_filtered['deleted_cx'].sum() / df_filtered['created_cx'].sum() * 100, 2)
    deleted_non_cx_ratio = round(df_filtered['deleted_non_cx'].sum() / df_filtered['created_non_cx'].sum() * 100, 2)
    
    if pr:
        print(f'During {period}, overall percentage of articles that were deleted,')
        print(f'\t- created using the Content Translation tool: {deleted_cx_ratio}%')
        print(f'\t- created without using the Content Translation Tool: {deleted_non_cx_ratio}%')
    else:
        return {
            'deleted_cx_pct': deleted_cx_ratio,
            'deleted_non_cx_pct': deleted_non_cx_ratio
        }

# calculate deletion (cx & non-cx) ratios by wiki for a given dataframe
def generate_ratios_by_wiki(df: pd.DataFrame, filter_threshold=15):
    df_filtered = df.query("created_cx > @filter_threshold")
    
    df_filtered = df_filtered.assign(
        deleted_cx_pct=round(df_filtered['deleted_cx'] / df_filtered['created_cx'] * 100, 2),
        deleted_non_cx_pct=round(df_filtered['deleted_non_cx'] / df_filtered['created_non_cx'] * 100, 2),
        deletion_pct_diff=lambda x: x['deleted_non_cx_pct'] - x['deleted_cx_pct']
    )
    
    df_filtered = df_filtered.set_index('wiki_db')
    
    return df_filtered

In [214]:
%%time

warnings.filterwarnings('ignore')

deletion_stats_currq_all = query_deletion_stats('current')
deletion_stats_prevq_all = query_deletion_stats('previous')

23/09/16 13:06:59 WARN SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.


CPU times: user 1.06 s, sys: 258 ms, total: 1.31 s
Wall time: 12min 57s


# Analysis

## Current-Quarter

In [226]:
# overal deletion percent
overall_deletion_pct(deletion_stats_currq_all, 'FY23-Q4', -1, True)

During FY23-Q4, overall percentage of articles that were deleted,
	- created using the Content Translation tool: 3.99%
	- created without using the Content Translation Tool: 4.06%


In [227]:
# deletion ratio by wiki
deletion_stats_currq = generate_ratios_by_wiki(deletion_stats_currq_all)

In [229]:
print(f'During FY23-Q4, acorss all wikis where more than 15 articles have been created with the Content Translation tool,\n\
there were {deletion_stats_currq.query("""deletion_pct_diff < 0""").shape[0]} wikis where articles created using CX \
were deleted more than articles created without using CX')

During FY23-Q4, acorss all wikis where more than 15 articles have been created with the Content Translation tool,
there were 23 wikis where articles created using CX were deleted more than articles created without using CX


In [230]:
# wikis with high deletion ratio
currq_high_deletion_ratio = deletion_stats_currq.query("""deletion_pct_diff < 0""").sort_values('deletion_pct_diff')
currq_high_deletion_ratio

,created_cx,created_non_cx,deleted_cx,deleted_non_cx,deleted_cx_pct,deleted_non_cx_pct,deletion_pct_diff
wiki_db,,,,,,,
shwiki,20,791,13,54,65.00,6.83,-58.17
suwiki,19,144,10,15,52.63,10.42,-42.21
tnwiki,32,32,14,4,43.75,12.50,-31.25
lvwiki,34,3521,10,225,29.41,6.39,-23.02
ltwiki,59,4077,17,574,28.81,14.08,-14.73
fiwiki,76,7945,17,765,22.37,9.63,-12.74
gdwiki,17,90,2,3,11.76,3.33,-8.43
iuwiki,16,12,16,11,100.00,91.67,-8.33
ttwiki,73,531,11,37,15.07,6.97,-8.10


## Previous-Quarter

In [231]:
# overal deletion percent
overall_deletion_pct(deletion_stats_prevq_all, 'FY23-Q4', -1, True)

During FY23-Q4, overall percentage of articles that were deleted,
	- created using the Content Translation tool: 4.03%
	- created without using the Content Translation Tool: 5.77%


In [233]:
# deletion ratio by wiki
deletion_stats_prevq = generate_ratios_by_wiki(deletion_stats_prevq_all)

In [196]:
print(f'During FY23-Q4, acorss all wikis where more than 15 articles have been created with the Content Translation tool,\n\
there were {deletion_stats_prevq.query("""deletion_pct_diff < 0""").shape[0]} wikis where articles created using CX \
were deleted more than articles created without using CX')        

During FY23-Q4, acorss all wikis where more than 15 articles have been created with the Content Translation tool,
there were 36 wikis where articles created using CX were deleted more than articles created without using CX


In [234]:
# wikis with high deletion ratio

prevq_high_deletion_ratio = deletion_stats_prevq.query("""deletion_pct_diff < 0""").sort_values('deletion_pct_diff')
prevq_high_deletion_ratio

,created_cx,created_non_cx,deleted_cx,deleted_non_cx,deleted_cx_pct,deleted_non_cx_pct,deletion_pct_diff
wiki_db,,,,,,,
bowiki,17,98,15,6,88.24,6.12,-82.12
iuwiki,202,44,202,20,100.00,45.45,-54.55
htwiki,84,91,35,3,41.67,3.30,-38.37
gdwiki,18,24,9,3,50.00,12.50,-37.50
jvwiki,83,295,32,15,38.55,5.08,-33.47
lawiki,17,642,5,40,29.41,6.23,-23.18
crwiki,36,24,36,19,100.00,79.17,-20.83
yiwiki,31,68,16,21,51.61,30.88,-20.73
iswiki,25,1451,5,50,20.00,3.45,-16.55


In [236]:
# wikis that had high deletion rates for articles that have been created with CX compared articles that have not been created using CX
wikis_high_deletion_ratio = np.intersect1d(currq_high_deletion_ratio.index.values, prevq_high_deletion_ratio.index.values)
wikis_high_deletion_ratio

array(['afwiki', 'bswiki', 'gdwiki', 'hywiki', 'iuwiki', 'kswiki',
       'kuwiki', 'ltwiki', 'ttwiki', 'tumwiki'], dtype=object)

## Formatting
for publication on Meta-Wiki at [Content translation/Deletion statistics comparison](https://www.mediawiki.org/wiki/Content_translation/Deletion_statistics_comparison)

In [305]:
currq_wtable = currq_high_deletion_ratio.copy()
percentage_columns = ['deleted_cx_pct', 'deleted_non_cx_pct', 'deletion_pct_diff']
currq_wtable[percentage_columns] = currq_wtable[percentage_columns]/100

In [302]:
def format_percent(column, df=currq_wtable):
    return df[column].map('{:.2%}'.format)

In [306]:
currq_wtable = (
    currq_wtable
    .assign(
        deleted_cx_pct = format_percent('deleted_cx_pct'),
        deleted_non_cx_pct = format_percent('deleted_non_cx_pct'),
        deletion_pct_diff = format_percent('deletion_pct_diff')
    )
    .reset_index()
)

currq_wtable.head()

,wiki_db,created_cx,created_non_cx,deleted_cx,deleted_non_cx,deleted_cx_pct,deleted_non_cx_pct,deletion_pct_diff
0,shwiki,20,791,13,54,65.00%,6.83%,-58.17%
1,suwiki,19,144,10,15,52.63%,10.42%,-42.21%
2,tnwiki,32,32,14,4,43.75%,12.50%,-31.25%
3,lvwiki,34,3521,10,225,29.41%,6.39%,-23.02%
4,ltwiki,59,4077,17,574,28.81%,14.08%,-14.73%


In [307]:
columns_rename_map = {
    'wiki_db': 'Wiki project',
    'created_cx': 'Created CX Articles', 
    'created_non_cx': 'Created non-CX Articles', 
    'deleted_cx': 'Deleted CX Articles', 
    'deleted_non_cx': 'Deleted non-CX Articles',
    'deleted_cx_pct': 'CX Articles Deletion Ratio', 
    'deleted_non_cx_pct': 'Non-CX Articles Deletion Ratio', 
    'deletion_pct_diff': 'Deletion Ratio Difference'
}

currq_wtable.rename(columns_rename_map, axis=1, inplace=True)

In [319]:
currq_wtable.columns = pd.MultiIndex.from_arrays(column_arrays)

In [320]:
currq_wtable

Wiki project    Created Articles                          \
   Wiki project Created CX Articles Created non-CX Articles   
0        shwiki                  20                     791   
1        suwiki                  19                     144   
2        tnwiki                  32                      32   
3        lvwiki                  34                    3521   
4        ltwiki                  59                    4077   
5        fiwiki                  76                    7945   
6        gdwiki                  17                      90   
7        iuwiki                  16                      12   
8        ttwiki                  73                     531   
9        kuwiki                  86                     568   
10       bswiki                 110                     785   
11       hywiki                 191                    2897   
12       ocwiki                  48                     930   
13       bhwiki                  68                     296   
14      tumwiki                  56                    5442   
15       mtwiki                  59                     194   
16       zhwiki                1905                   40421   
17       kswiki                 145                    1627   
18       afwiki                  98                    1573   
19       glwiki                 504                    2367   
20       dewiki                 595                   50390   
21       bewiki                 604                    3745   
22       jawiki                 683                   20939   

      Deleted Articles                                    Deletion Ratios  \
   Deleted CX Articles Deleted non-CX Articles CX Articles Deletion Ratio   
0                   13                      54                     65.00%   
1                   10                      15                     52.63%   
2                   14                       4                     43.75%   
3                   10                     225                     29.41%   
4                   17                     574                     28.81%   
5                   17                     765                     22.37%   
6                    2                       3                     11.76%   
7                   16                      11                    100.00%   
8                   11                      37                     15.07%   
9                    9                      23                     10.47%   
10                  22                     110                     20.00%   
11                  25                     263                     13.09%   
12                   3                      23                      6.25%   
13                  17                      66                     25.00%   
14                   3                     165                      5.36%   
15                   4                       9                      6.78%   
16                 140                    2273                      7.35%   
17                   2                       6                      1.38%   
18                   7                      97                      7.14%   
19                  21                      89                      4.17%   
20                  89                    7346                     14.96%   
21                  16                      95                      2.65%   
22                  89                    2720                     13.03%   

                                                             
   Non-CX Articles Deletion Ratio Deletion Ratio Difference  
0                           6.83%                   -58.17%  
1                          10.42%                   -42.21%  
2                          12.50%                   -31.25%  
3                           6.39%                   -23.02%  
4                          14.08%                   -14.73%  
5                           9.63%                   -12.74%  
6    